# IEL DATA EXPLORATION
This Jupiter notebook is used to explore the data and record my thought process of how I will approach the problem. Early on in the project I changed data sources from initially using multiple sources (NRFA, River Levels UK, SEPA) however, I changed this so all data is coming from SEPA's API as this contains a larger volume of data and is more accurate. However, because of this change some observations which I made in the previous dataset are saved in the OLD DATA notebook. I have copied the relevant sections from the old notebook to this one so that I can refer back to them if needed.

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import os
import sklearn

### Read in the data
Currently we are using data from the "Carradale Water at Dippen" Water Station. This is a water station which is located in Cambeltown, on the West Coast of Scotland. The reason for choosing this station is because it has approximately the same latitude, and geography as the fictitious island of Claddach and monitors a river pretty much all the way from the source to the river estuary. This means that the data should be representative of the data we would expect to see on Claddach.

See the README.md in the Data repository to see how this data was collected.

In [ ]:
# Import the new data and create a dataframe
daily_flow = pd.read_csv('Data/Daily_Averages/Daily_Mean_Flow_Rate.csv', delimiter=';')
daily_flow.rename(columns={'Value': 'Mean Flow (m3/s)'}, inplace=True)
daily_rain = pd.read_csv('Data/Daily_Averages/Daily_Precipitation.csv', delimiter=';')
daily_rain.rename(columns={'Value': 'Daily Precipitation (mm)'}, inplace=True)
daily_level = pd.read_csv('Data/Daily_Averages/Daily_Mean_Level.csv', delimiter=';')
daily_level.rename(columns={'Value': 'Mean Water Level (m)'}, inplace=True)

In [ ]:
# Merge the 3 datasets into one
merged = pd.merge(daily_flow, daily_rain, on=['#Timestamp'])
merged = pd.merge(merged, daily_level, on=['#Timestamp'])
merged = merged.drop('Quality Code_x', axis=1)
merged = merged.drop('Quality Code_y', axis=1)
merged = merged.drop('Quality Code', axis=1)
merged.dropna(inplace=True)
merged.describe()

### Simple Calculations

The Catchment Area of the Carradale Water at Dippen Water Station is 58.5km^2 (https://www2.sepa.org.uk/waterlevels/default.aspx?sd=t&lc=133077).
<br>1mm of rainfall is equal to 1 litre of water per square metre.

The Scottish Governments states that the estimated percentage of precipitation which runoffs into the see is approximately 73% (https://www.gov.scot/publications/scotlands-marine-atlas-information-national-marine-plan/pages/7/) However, this is extremely variable from location to location (eg. in the USA this is as low as 30% (https://www.usgs.gov/special-topics/water-science-school/science/rain-and-precipitation). As well as the fact that on a daily, or even hourly basis, this will depend upon the weather conditions as more sunshine and warmer weather will lead to more evaporation and less ending up in the river. Therefore for now, we will set the percentage of rainfall which ends up in the river to a figure which results in the cumulative water difference throughout the year being equal to 0. This will likely be changed later to a more realistic value which changes throughout the year to account for the weather conditions.

<br>hence the amount of water going into our river (measured in litres) on a given day is:
    <br><br> rainfall * 58.5 * 0.73 *1,000,000
    <br>     rainfall * 42,705,000

For the amount of water going out of the river (measured in litres) on a given day, we can use the flow data.
<br><br>We know that the flow data is measured in m^3/s, so we can convert this to litres (by multiplying by 1000) and then multiply by the number of seconds in a day (86400) to get the litres of water going out of the river on a given day.

<br>flow * 1000 * 86400
<br>flow * 86,400,000

<br>Hence the water difference on a given day is equal to:
<br><br>rainfall * 42,705,000 - flow * 86,400,000

We want to convert this back into cubic metres, so we divide by 1000
<br><br>rainfall * 42,705 - flow * 86,400

Let's add this to our dataframe and plot it.


In [ ]:
waterDifference = merged['Daily Precipitation (mm)']*45000 - merged['Mean Flow (m3/s)']*86400
merged['Water Difference (m3)'] = waterDifference
merged.describe()

In [ ]:
# Calculate the derivative of the water level for each day. Don't ask why it's calculated this way - Python was just being a pain.
levelDerivative = []
previous = merged['Mean Water Level (m)'][0]
flag = True
for i in merged['Mean Water Level (m)']:
    diff = i - previous
    if not flag:
        levelDerivative.append(diff)
    flag = False
    previous = i
levelDerivative.append(0)
merged['level_derivative'] = levelDerivative
merged

In [ ]:
# Plot the water difference and the level derivative against the date with different axis scales
fig, ax1 = plt.subplots()
ax1.plot(merged['#Timestamp'][365:730], merged['Water Difference (m3)'][365:730])
ax1.set_xlabel('Date')
ax1.set_ylabel('Water Difference', color='tab:blue')
ax1.set_ylim(-1000000, 1000000)
ax2 = ax1.twinx()
ax2.plot(merged['#Timestamp'][365:730], merged['level_derivative'][365:730], color='tab:red')
ax2.set_ylabel('Level Derivative', color='tab:red')
ax2.set_ylim(-1, 1)
plt.show()

In [ ]:
#cols = ['Mean Flow (m3/s)', 'Daily Precipitation (mm)', 'Mean Water Level (m)', 'Water Difference (m3)', 'level_derivative']
#sns.pairplot(merged[cols], height=2.5)

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(merged[cols].values)
cov_mat = np.cov(X_std.T)
plt.figure(figsize=(10, 10))
hm = sns.heatmap(cov_mat, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 15}, cmap='coolwarm', yticklabels=cols, xticklabels=cols)
plt.title('Covariance Matrix')
plt.tight_layout()
plt.show()

From the above covariance matrix - it should be possible to model the water flow and level using only the rainfall data. Since the water_difference for the day doesn't correlate too much with the mean flow and since the mean flow correlates with the water level.

Hence we can model it as follows:

Rainfall -> Water Difference -> x -> Water Level -> Water Flow -> Level Derivative

In [ ]:

# Fit a polynomial curve to the data
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

X = merged['Mean Water Level (m)'].values.reshape(-1, 1)
y = merged['Mean Flow (m3/s)'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quadratic = PolynomialFeatures(degree=4)
X_quad = quadratic.fit_transform(X)
X_quad_test = quadratic.fit_transform(X_test)
lr = LinearRegression()
lr.fit(X_quad, y)
y_quad_fit = lr.predict(X_quad)
y_quad_pred = lr.predict(X_quad_test)
print('MSE train: %.3f, test: %.3f' % (
mean_squared_error(y, y_quad_fit),
mean_squared_error(y_test, y_quad_pred)))
print('R^2 train: %.3f, test: %.3f' % (
r2_score(y, y_quad_fit),
r2_score(y_test, y_quad_pred)))

# Plot the polynomial curve
plt.scatter(X, y, label='data points', color='lightgray', marker='o')

x = np.linspace(0, 2.7, 1000)
y = lr.predict(quadratic.fit_transform(x.reshape(-1, 1)))
plt.plot(x, y, label='quadratic (d=2)', color='red')

plt.legend(loc='upper left')
plt.show()

In [ ]:
# Remove days with water difference > 2000000 and > 2000000
typical = merged[merged['Water Difference (m3)'] < 2000000]
typical = typical[typical['Water Difference (m3)'] > -2000000]

print(len(typical))
print(len(typical) / len(merged))

# Fit a polynomial curve of Water difference (independent) and level derivative (dependent)
X = typical['Water Difference (m3)'].values.reshape(-1, 1)
y = typical['level_derivative'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quadratic = PolynomialFeatures(degree=1)
X_quad = quadratic.fit_transform(X)
X_quad_test = quadratic.fit_transform(X_test)
lr = LinearRegression()
lr.fit(X_quad, y)
y_quad_fit = lr.predict(X_quad)
y_quad_pred = lr.predict(X_quad_test)

# Plot the polynomial curve
plt.scatter(X, y, label='data points', color='lightgray', marker='.')

x = np.linspace(min(X), max(X), 1000)
y = lr.predict(quadratic.fit_transform(x.reshape(-1, 1)))
plt.plot(x, y, label='predicted', color='red')

plt.xlim(-2000000, 2000000)
plt.ylim(-1, 2)

plt.legend(loc='upper left')
plt.title('Regression of Flow Rate against Water Level')
plt.xlabel('Water Difference (m3)')
plt.ylabel('Level Derivative (m)')
plt.show()

# Calculate the standard deviation of the residuals
residuals = y_quad_pred - y_test
print('Standard Deviation of Residuals: %.3f' % np.std(residuals))

# Calculate the standard deviation of the water difference
print('Standard Deviation of Water Difference: %.3f' % np.std(X_test))

# Calculate the mean water difference
print('Mean Water Difference: %.3f' % np.mean(X_test))


In [ ]:
# Plot a histogram of the water difference
plt.hist(X_test, bins=100)
plt.title('Histogram of Water Difference')
plt.xlabel('Water Difference (m3)')
plt.ylabel('Frequency')
plt.show()

# Plot a histogram of the residuals
plt.hist(residuals, bins=100)
plt.title('Histogram of Residuals')
plt.xlabel('Residuals (m3)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Draw from a gaussian distribution with the mean and standard deviation of the water difference
# and the standard deviation of the residual
import random
import math
import matplotlib.pyplot as plt
import numpy as np

dataPoints = [(0, 0)] * 9370
for i in range(9370):
    x = random.choice(X_test)[0]
    temp = lr.predict(quadratic.fit_transform(x.reshape(-1, 1)))
    y = random.gauss(temp, np.std(residuals))
    dataPoints[i] = (x,y)


    
# Plot the data points
plt.scatter(*zip(*dataPoints), label='data points', color='lightgray', marker='.')
plt.xlim(-2000000, 2000000)
plt.ylim(-1, 2)

In [ ]:
# Plot the water difference against the precipitation
plt.scatter(typical['Daily Precipitation (mm)'], typical['Water Difference (m3)'], label='data points', color='lightgray', marker='.')
plt.show()

In [ ]:
# Using linear regression to predict the water difference
X = typical['Daily Precipitation (mm)'].values.reshape(-1, 1)
y = typical['Water Difference (m3)'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Plot the polynomial curve
plt.scatter(X, y, label='data points', color='lightgray', marker='.')

x = np.linspace(0, 60, 1000)
y = lr.predict(x.reshape(-1, 1))
plt.plot(x, y, label='predicted', color='red')

plt.xlabel('Precipitation (mm)')
plt.ylabel('Water Difference (m3)')
plt.show()
    